In [2]:
from cybershuttle_tune.sdk import ExecutionContext
from cybershuttle_tune.sdk import ApplicationContext
from cybershuttle_tune.sdk import TuneConfig
from cybershuttle_tune.sdk import DiscreteParam
from cybershuttle_tune.sdk import run_grid_search
from cybershuttle_tune.sdk import get_sweep_status
from cybershuttle_tune.sdk import fetch_outputs
from cybershuttle_tune.sdk import get_remote_data_dirs
from cybershuttle_tune.sdk import authorize
from cybershuttle_tune.cli.auth import get_access_token
import os
import json
from pathlib import Path

using legacy validation callback
/usr/local/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/local/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [3]:
import numpy as np

In [4]:
# Review Input Parameters
! cat inputs/network.cfg


 [Types]
#9  types
1 1 #100% - (float) ratios, how to multiply the x & y numbers below if needed
#name x     y (both can be floats)
 RE   100  1  #0 
 REa    0  0 #30  1  #1 
 TC   100  1  #2 
 TCa    0  0 #30  1  #3 
 CX   500  1 #100  1  #4 
 CXa    0  0 #100  1 #100  1  #5 
 CX6    0  0 #100  1 #100  1  #8 
 IN   100  1 #25  1  #6 
 INa    0  0 #25  1 # 25  1  #7 
 IN6    0  0 #25  1 # 25  1  #9 

#Short range connections
#Radius is from the output layer POV
#distribution type left for future purposes, e.g. uniform gauss
# AMPA_D1 <-=-> TC<->CX,  AMPA <-=-> TC->RE,  AMPA_D2 <-=-> CX->CX, GABA_A <-=-> RE->TC, GABA_A_D2 <-=-> IN->CX
[Connections] 
  #From  To   Type   Radius  ColSci  Prob  Prob_oc  Distrib    Strength      Mini_s  Mini_f      
  RE     TC   GABA_A      8   10000    1    1  fixed    {{ re_tc }}         0.0     0.0  # 0 
  RE     TC   GABA_B      8   10000    1    1  fixed    0.00000100         0.0     0.0      
  RE     RE   GABA_A      5   10000    1    1  fixed    0

In [5]:
import sys
sys.path.append('/')
import airavata_magics
airavata_magics.load_ipython_extension(get_ipython())

In [6]:
# Launch 10 simulations with paramaterization of histamine from 0.3 to 1.2

#tc_re = np.arange(0.000010000,0.00010000,0.000010000)
tc_re = [0.00005000]
re_tc = np.arange(0.000010000,0.00010000,0.000010000)
in_cx = np.arange(0.000015000,0.00040000,0.000025000)

in_cx_mini = [0.00100000]

params = [
    DiscreteParam('re_tc', re_tc),
    DiscreteParam('tc_re', tc_re),
    DiscreteParam('in_cx', in_cx),
    DiscreteParam('in_cx_mini', in_cx_mini),
    
]

# Run the simulation on the ACCESS-CI, San Diego Supercomputer Centers Expanse Cluster
execution_context = ExecutionContext(resource = "Expanse", 
                                     project = "Default Project", 
                                     group_resource_profile = "Default", 
                                     cpu = 1, 
                                     memory = 1000, 
                                     queue = "shared")

input_file_mapping = {"Network Config File":"network.cfg", "Param File": "params.txt"}
app_context = ApplicationContext(app_name = "Sleep-Stage-Transition", input_dir = "inputs", input_file_mapping = input_file_mapping)

tune_config = TuneConfig(
    app_context = app_context,
    params = params, 
    execution_context = execution_context)

In [7]:
# Launch the paramater sweep runs on remote HPC Resourcs
response = run_grid_search(tune_config = tune_config)

/usr/local/lib/python3.12/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


{'Network Config File': 'airavata-dp://ded6d8b1-76bd-499b-9a49-2e249f25c7e5', 'Param File': 'airavata-dp://f3c34fd2-1c25-4724-9b12-0b6ec6c319da'}
Airavata experiment 560hGM_33_93c75d89-4e01-4820-8344-5271df0d5a14 was launched for sweep config 33
{'Network Config File': 'airavata-dp://517480ae-56e9-4eb7-b3b7-43d7d81bb97d', 'Param File': 'airavata-dp://36e4690a-a636-4314-8386-b6c200de0ac3'}
Airavata experiment 560hGM_107_fd131e1e-2dd7-4693-8b0c-7182dbe43aff was launched for sweep config 107
{'Network Config File': 'airavata-dp://e01165bb-3c0b-4866-8352-c2380234190e', 'Param File': 'airavata-dp://a4dfc1b5-ec93-48e6-a049-1687d0fd0d59'}
Airavata experiment 560hGM_106_fee83a00-7500-4678-981c-a76eeb8bdb2a was launched for sweep config 106
{'Network Config File': 'airavata-dp://e591bf6e-e43e-44ec-8118-67bbd4857285', 'Param File': 'airavata-dp://c8ec907f-ae6c-4a87-8941-6da7497c8c08'}
Airavata experiment 560hGM_68_01cc6b18-abee-4dc5-9e5a-4030a3474ef3 was launched for sweep config 68
{'Network Co

In [8]:
# Query Cybeshuttle Server to fetch the job status. Wait until all statuses are complete
states, indexes = get_sweep_status(response['job_name'], response['working_dir'])
states

/usr/local/lib/python3.12/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


[['560hGM_33_93c75d89-4e01-4820-8344-5271df0d5a14', 'COMPLETED'],
 ['560hGM_107_fd131e1e-2dd7-4693-8b0c-7182dbe43aff', 'COMPLETED'],
 ['560hGM_106_fee83a00-7500-4678-981c-a76eeb8bdb2a', 'COMPLETED'],
 ['560hGM_68_01cc6b18-abee-4dc5-9e5a-4030a3474ef3', 'COMPLETED'],
 ['560hGM_102_f18d116c-0b4d-4393-892e-1d3a1170075e', 'COMPLETED'],
 ['560hGM_39_88cc7966-8b8b-4982-9468-36b7938b023d', 'COMPLETED'],
 ['560hGM_26_2cfa3cc4-62a2-4e2a-9580-6e877aacff63', 'COMPLETED'],
 ['560hGM_64_e7ddefa6-231d-4fe5-8252-ebfe36649ad8', 'COMPLETED'],
 ['560hGM_84_a7d00fa8-40ae-487e-ab26-2863ed771784', 'COMPLETED'],
 ['560hGM_87_9767cdc2-97e9-4c17-b618-d3b971d267bf', 'COMPLETED'],
 ['560hGM_128_b67a8b8f-92d0-418d-b413-28e3f3cb7b76', 'COMPLETED'],
 ['560hGM_62_4d0d2e27-2026-41a9-98fa-193f6156705b', 'COMPLETED'],
 ['560hGM_30_bd73e49b-3644-4a8b-a21f-298bb7f8c10d', 'COMPLETED'],
 ['560hGM_92_f8429a3b-9f29-4139-ad9b-300e1e1ef7e9', 'EXECUTING'],
 ['560hGM_16_6a297a11-1e4f-4d8b-baa6-d6058d6ba8c8', 'EXECUTING'],
 ['560

In [9]:
remote_output_paths = get_remote_data_dirs(response['job_name'], response['working_dir'])

/usr/local/lib/python3.12/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


In [6]:
%init_remote cluster=expanse cpu=2 memory=2024 queue=shared walltime=60

Response: {'agentId': 'agent_275ef044-7407-4eb7-9ece-f2572b7d3617', 'experimentId': 'CS_Agent_7a9fa48f-cecd-462f-85d3-0824f08e294a'}


In [9]:
%status_remote

Agent agent_275ef044-7407-4eb7-9ece-f2572b7d3617 is still prepairing. Please wait


In [10]:
remote_output_paths = get_remote_data_dirs(response['job_name'], response['working_dir'])

NameError: name 'response' is not defined